In [1]:
from utils.structure import *
from utils.predicter import *
from utils.solver import *

In [2]:
# build structure and display
test_datas = open('./data/test_demo.txt', 'r')
test_datas = test_datas.readlines()
for test_data in test_datas:
    s = Structure(test_data)
#     print(s.data)
#     print(s.display())

In [3]:
s = Structure(test_datas[5])
print(s.display())

solver = BasicSolver(s)
# solver.idxes_need_to_solve

+-----+-----+-----+
|3 8 .|. . 7|. . .|
|     |     |     |
|. . 1|. 5 8|. . .|
|     |     |     |
|9 5 .|3 . .|. 1 8|
+-----+-----+-----+
|1 2 9|. 8 .|. 6 .|
|     |     |     |
|. . 8|. . .|. . .|
|     |     |     |
|. 3 .|. 9 .|. 8 2|
+-----+-----+-----+
|4 9 5|8 3 6|. . 1|
|     |     |     |
|8 . 3|5 7 .|6 . .|
|     |     |     |
|. . .|4 . 9|8 3 5|
+-----+-----+-----+



In [4]:
# solver.check_idx_only(62)
# print(solver.ready)
# solver.check_idx_last_left(62)
# print(solver.ready)
print(solver.check_scanned_drop('2'))
print(solver.check_area_drop('2'))
solver.ready

+-----+-----+-----+
|3 8 .|. . 7|. .  |
|     |     |     |
|.   1|. 5 8|. .  |
|     |     |     |
|9 5 .|3 . .|. 1 8|
+-----+-----+-----+
|1 2 9|  8  |  6  |
|     |     |     |
|    8|. . .|     |
|     |     |     |
|  3  |  9  |  8 2|
+-----+-----+-----+
|4 9 5|8 3 6|. . 1|
|     |     |     |
|8   3|5 7 .|6 .  |
|     |     |     |
|.   .|4 . 9|8 3 5|
+-----+-----+-----+

False
+-----+-----+-----+
|3 8 .|. . 7|. .  |
|     |     |     |
|.   1|. 5 8|. .  |
|     |     |     |
|9 5 .|3 . .|. 1 8|
+-----+-----+-----+
|1 2 9|  8  |  6  |
|     |     |     |
|    8|     |     |
|     |     |     |
|  3  |  9  |  8 2|
+-----+-----+-----+
|4 9 5|8 3 6|. . 1|
|     |     |     |
|8   3|5 7 .|6 .  |
|     |     |     |
|     |4   9|8 3 5|
+-----+-----+-----+

True


[(9, '2'), (68, '2')]

In [7]:
type(['', 1]) == list

True

In [8]:
{1, 2, 3} & {1, 2} & {2, 4}

{2}

In [9]:
tmp = "3	8	 	 	 	7	 	 	 	 	×	1	 	5	8	 	 	 	9	5	 	3	 	 	 	1	8	1	2	9	 	8	 	 	6	 	 	×	8	 	 	 	 	 	 	 	3	 	 	9	 	 	8	2	4	9	5	8	3	6	 	 	1	8	×	3	5	7	 	6	 	 	2	×	2	4	×	9	8	3	5"

In [10]:
tmp = tmp.replace(' ', '.').replace('×', '.').replace('	', '')
print(len(','.join(list(tmp))))
','.join(list(tmp))

161


'3,8,.,.,.,7,.,.,.,.,.,1,.,5,8,.,.,.,9,5,.,3,.,.,.,1,8,1,2,9,.,8,.,.,6,.,.,.,8,.,.,.,.,.,.,.,3,.,.,9,.,.,8,2,4,9,5,8,3,6,.,.,1,8,.,3,5,7,.,6,.,.,2,.,2,4,.,9,8,3,5'